In [1]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [5]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

        

Enter a class to grade (i.e. class1 for class1)
Successfully opened class7.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [7]:
def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
for grade in grades: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                incorrect_id = value[0]
                print("Invalid line of data:", (incorrect_id) + " does not contain exactly 26 values:")
                print(grade)    
values_count = list(map(valueCheck,grades))
values_check = list(map(has26Values,values_count))
print("Total valid lines of data:", values_check.count(True))
print("Total invalid lines of data:", values_check.count(False))


Invalid line of data: N00001966 does not contain exactly 26 values:
N00001966,B,A,,A,C,B,D,A,C,C,D,B,D,B,A,C,B,,A,C,D,A,A,

Invalid line of data: N00001974 does not contain exactly 26 values:
N00001974,B,D,D,D,C,A,D,A,A,C,B,B,C,B,A,C,B,D,D,D,A,A,B,D,D,A,A

Invalid line of data: N00002023 does not contain exactly 26 values:
N00002023,B,A,D,D,,B,D,A,D,,D,B,A,A,A,C,B,D,A,C,,C

Invalid line of data: N00002034 does not contain exactly 26 values:
N00002034,B,A,D,D,C,D,D,B,,C,D,A,A,B,A,C,B,C,A,C,,A,B,D,D,B

Total valid lines of data: 196
Total invalid lines of data: 4


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html
https://linuxhint.com/pandas-compare-two-dataframes-row-by-row/

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'